In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
from mlxtend.data import loadlocal_mnist
from pathlib import Path

In [ ]:
def min_max_normalizer(images):
    '''
        Min-Max normalizer for images
    '''
    images[:,0,:,:] = (images[:,0,:,:]-np.min(images[:,0,:,:]))/(images[:,0,:,:].max()-images[:,0,:,:].min())
    images[:,1,:,:] = (images[:,1,:,:]-np.min(images[:,1,:,:]))/(images[:,1,:,:].max()-images[:,1,:,:].min())
    images[:,2,:,:] = (images[:,2,:,:]-np.min(images[:,2,:,:]))/(images[:,2,:,:].max()-images[:,2,:,:].min())
    return images

In [ ]:
class Convolution_Layer():
    def __init__(self, input, filter_size, bias=True, stride=1, padding=0, dilation=1):
        '''
            Initializing a Conv layer with padding, strides and dilation options
        '''
        self.padding = padding
        self.stride = stride
        self.dilation = dilation

        input = input.reshape(len(input), 3, 32, 32)

        self.N, self.C, self.H, self.W = input.shape
        self.F, self.C, self.HH, self.WW = filter_size

        self.H_ = int((self.H + 2*padding - (dilation)*(self.HH-1)-1)/stride) + 1
        self.W_ = int((self.W + 2*padding - (dilation)*(self.WW-1)-1)/stride) + 1        
        
        self.weights = np.random.normal(loc=0,scale=1, size=filter_size)
        self.output = np.zeros((self.N, self.F, self.H_, self.W_))


    def forward(self,input):
        pad = self.padding
        input = input.reshape(len(input), 3, 32, 32)
        self.input = np.pad(input, ((0,0),(0,0),(pad,pad),(pad,pad)))

        # Loop over every location in inp_height * inp_width for the whole batch
        j_HH = (self.dilation)*(self.HH-1)+1
        j_WW = (self.dilation)*(self.WW-1)+1
        
        d_weigths = np.zeros((self.F, self.C, j_HH, j_WW))
        for f in range(self.F):
            for c in range(self.C):
                for hh in range(j_HH):
                    if(hh%self.dilation)==0:
                        for ww in range(j_WW):
                            if(ww%self.dilation)==0:
                                d_weigths[f,c,hh,ww] = self.weights[f,c,int(hh/self.dilation), int(ww/self.dilation)]
        
        self.weights = d_weigths
      
        for n in range(self.N):
            for f in range(self.F):
                for h in range(self.H_):
                    for w in range(self.W_):
                        self.output[n,f,h,w] = np.sum(self.input[n,
                                                                 :,
                                                                 h*self.stride:h*self.stride+j_HH,
                                                                 w*self.stride:w*self.stride+j_WW] * d_weigths[f,:,:,:])
        # Output will be of the size (Batch_size, out_channels, out_height, out_width)
        return self.output

    def backward(self, grad_of_output_size):
        # Naive Implementation
        j_HH = (self.dilation)*(self.HH-1)+1
        j_WW = (self.dilation)*(self.WW-1)+1
        
        grad = np.zeros_like(self.weights)
        for n in range(self.N):
            for f in range(self.F):
                for hi in range(self.H_):
                    for wi in range(self.W_):
                        grad[f] += self.input[n, :, hi*self.stride:hi*self.stride+j_HH, wi*self.stride:wi*self.stride+j_WW] * grad_of_output_size[n, f, hi, wi]

        return grad

    def set_weights(self, new_weights):
        self.weights = new_weights
        
    def update_weights(self, lr, grad):
        self.weights = self.weights - lr*grad


In [ ]:
data_batch = []
label_batch = []
for i in range(5):
    file = f'cifar10/data_batch_{i+1}'
    def unpickle(file):
        import pickle
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    data_batch_ = unpickle(file)
    data_batch.append(data_batch_[b'data'])
    label_batch.append(data_batch_[b'labels'])

# Test data    
file = f'cifar10/test_batch'
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
test_data = unpickle(file)[b'data']
test_labels = unpickle(file)[b'labels']

In [ ]:
filter_size = (3,3,3,3)

def gaussian(i, sigma):
	G = (np.exp(-0.5*((i)/sigma)**2))/(math.pi*2*sigma**2)
	return G

new_weights = np.zeros(filter_size)
for i in range(filter_size[0]):
    for j in range(filter_size[1]):
        for k in range(filter_size[2]):
            for l in range(filter_size[3]):
                new_weights[i,j,k,l] = gaussian((k-(filter_size[2]//2))**2 + (l-(filter_size[3]//2))**2, i*0.2+1)
        



In [ ]:
class L2_loss():
    def __init__(self):
        pass
    def forward(self, C0_output,C_output):
        loss = np.sum(np.square(C0_output-C_output))
        return loss
    
    def backward(self, C0_output,C_output):
        grad = 2 * (C0_output - C_output)
        return grad


In [ ]:
l2_loss = L2_loss()

filter_size = (20,3,5,5)
new_weights = np.random.uniform(-1,1,size=filter_size)

inputs = data_batch[0][:100]

conv = Convolution_Layer(inputs,filter_size=filter_size, bias=True, stride=1, padding=0, dilation=1)
conv.set_weights(new_weights)
C_output = conv.forward(inputs)

In [ ]:
EPOCHS = 10
lr = 3e-3

losses = []

for e in range(EPOCHS):
    C_output = conv.forward(inputs)
    C_output = min_max_normalizer(C_output)
    loss = l2_loss.forward(C0_output, C_output)
    losses.append(loss)
    print("EPOCH : ",e, "LOSS : ",loss)
    loss_grad = l2_loss.backward(C0_output,C_output)
    grad = conv.backward(loss_grad)
    conv.update_weights(lr, grad)
    